<a href="https://colab.research.google.com/github/Alishaw99/Quantize_LLLMs/blob/main/Quantize_LLMs_to_GGUF_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/ggerganov/llama.cpp

Cloning into 'llama.cpp'...
remote: Enumerating objects: 30077, done.
remote: Counting objects: 100% (122/122), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 30077 (delta 58), reused 92 (delta 46), pack-reused 29955
Receiving objects: 100% (30077/30077), 54.56 MiB | 7.07 MiB/s, done.
Resolving deltas: 100% (21407/21407), done.


In [2]:
!cd llama.cpp && LLAMA_CUBLAS=1 make && pip install -r requirements/requirements-convert-hf-to-gguf.txt

Makefile:77: *** LLAMA_CUBLAS is removed. Use GGML_CUDA instead..  Stop.


In [3]:
from huggingface_hub import snapshot_download

In [4]:
model_name = "Qwen/Qwen1.5-1.8B"

In [5]:
methods = ['q4_k_m']

In [6]:
base_model = "./original_model/"
quantized_path = "./quantized_model/"

In [7]:
snapshot_download(repo_id=model_name, local_dir=base_model , local_dir_use_symlinks=False)
original_model = quantized_path+'/FP16.gguf'

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1194: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

LICENSE:   0%|          | 0.00/7.28k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.67G [00:00<?, ?B/s]

In [8]:
!python /content/llama.cpp/convert_hf_to_gguf.py ./original_model/ --outtype f16 --outfile .//quantized_model/FP16.gguf


INFO:hf-to-gguf:Loading model: original_model
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model part 'model.safetensors'
INFO:hf-to-gguf:output.weight,             torch.bfloat16 --> F16, shape = {2048, 151936}
INFO:hf-to-gguf:token_embd.weight,         torch.bfloat16 --> F16, shape = {2048, 151936}
INFO:hf-to-gguf:blk.0.attn_norm.weight,    torch.bfloat16 --> F32, shape = {2048}
INFO:hf-to-gguf:blk.0.ffn_down.weight,     torch.bfloat16 --> F16, shape = {5504, 2048}
INFO:hf-to-gguf:blk.0.ffn_gate.weight,     torch.bfloat16 --> F16, shape = {2048, 5504}
INFO:hf-to-gguf:blk.0.ffn_up.weight,       torch.bfloat16 --> F16, shape = {2048, 5504}
INFO:hf-to-gguf:blk.0.ffn_norm.weight,     torch.bfloat16 --> F32, shape = {2048}
INFO:hf-to-gguf:blk.0.attn_k.bias,         torch.bfloat16 --> F32, shape = {2048}
INFO:hf-to-gguf:blk.0.attn_k.weight,       torch.bfloat16 --> F16, shape = {2048, 2048}
INFO:hf-to-

In [9]:
!mkdir ./quantized_model/

In [10]:
import os

In [11]:
for m in methods:
    qtype = f"{quantized_path}/{m.upper()}.gguf"
    os.system("./llama.cpp/quantize "+quantized_path+"/FP16.gguf "+qtype+" "+m)

In [12]:
!./content/llama.cpp/main -m ./content/quantized_model/FP16.gguf -n 90 --repeat_penalty 1.0 --color -i -r "User:" -f llama.cpp/prompts/chat-with-bob.txt

/bin/bash: line 1: ./content/llama.cpp/main: No such file or directory


In [13]:

from huggingface_hub import notebook_login
notebook_login()

In [14]:
from huggingface_hub import HfApi, HfFolder, create_repo, upload_file

In [15]:
model_path = "./content/quantized_model/Q4_K_M.gguf" # Your model's local path
repo_name = "qwen1.5-llm"  # Desired HF Hub repository name
repo_url = create_repo(repo_name, private=False)

HfHubHTTPError: 409 Client Error: Conflict for url: https://huggingface.co/api/repos/create (Request ID: Root=1-669d924a-2c179f4417e27a1405e1417f;82b32cc0-b612-42c2-ac89-f863c6eb8322)

You already created this model repo

In [16]:
api = HfApi()
api.upload_file(
    path_or_fileobj=model_path,
    path_in_repo="Q4_K_M.gguf",
    repo_id="skuma307/qwen1.5-llm",
    repo_type="model",
)

ValueError: Provided path: 'content/quantized_model/Q4_K_M.gguf' is not a file on the local file system